### Importing the Libraries

In [24]:
import pandas as pd
import os
import sys
sys.path.append(os.path.abspath(".."))
import utils
from utils import helper_functions as utils

### Uploading the Data

In [25]:
filepath = '../data/events.csv' 
data = pd.read_csv(filepath) 
df = data.copy() 

Basic check on the what the data is and how it looks like

In [26]:
utils.view_data(df)

Data has 885129 rows and 9 columns with these data types:

event_time        object
event_type        object
product_id         int64
category_id        int64
category_code     object
brand             object
price            float64
user_id            int64
user_session      object
dtype: object

Data row sample and full columns:


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
399021,2020-12-05 10:48:49 UTC,cart,491348,2144415927611949295,electronics.audio.acoustic,NaN,9.92,1515915625548950294,PTLQeWRy6N
242470,2020-11-09 09:28:27 UTC,purchase,1521673,2144415922201296994,NaN,NaN,19.52,1515915625537334968,iKnIRIJLZG
813250,2021-02-16 07:38:03 UTC,view,1703409,2144415935698567627,NaN,mastech,66.19,1515915625606023491,pyzNKHDCso
850151,2021-02-22 13:35:51 UTC,view,4079110,2144415922427789416,computers.components.videocards,powercolor,464.52,1515915625604081922,ecW9JOAjES
730240,2021-02-02 19:01:09 UTC,view,879734,2144415922427789416,computers.components.videocards,gigabyte,108.63,1515915625599915370,R0MgXpNHYx


checking for duplications as well as empty or empty string rows

In [27]:
utils.check_data(df)

There are 655 duplicate rows, 448748 empty rows and 0 empty spaces
Duplicate Rows:
                     event_time event_type  product_id          category_id  \
511     2020-09-24 13:51:07 UTC       view      387956  2144415922427789416   
512     2020-09-24 13:51:07 UTC       view      387956  2144415922427789416   
974     2020-09-24 15:48:55 UTC       view      874667  2144415922738167921   
975     2020-09-24 15:48:55 UTC       view      874667  2144415922738167921   
4827    2020-09-25 13:15:09 UTC       view      453469  2144415924222951574   
...                         ...        ...         ...                  ...   
878751  2021-02-27 17:56:05 UTC       view     1571204  2144415924491387038   
879544  2021-02-27 20:54:20 UTC       view     1027953  2144415923837075596   
879545  2021-02-27 20:54:20 UTC       view     1027953  2144415923837075596   
882715  2021-02-28 14:18:02 UTC       view     4078916  2144415922427789416   
882716  2021-02-28 14:18:02 UTC       view     4

In [28]:
# Drop rows with NaN values
df = df.dropna()

# Drop duplicated rows
df = df.drop_duplicates()

# Display the updated dataframe shape
df.shape

(515855, 9)

making sure the dataset is clean

In [29]:
utils.check_data(df)

There are 0 duplicate rows, 0 empty rows and 0 empty spaces


# EDA

In [30]:
target = df.pop('event_type')
df['event_type'] = target

In [31]:
df.dtypes

event_time        object
product_id         int64
category_id        int64
category_code     object
brand             object
price            float64
user_id            int64
user_session      object
event_type        object
dtype: object

From looking at the datatypes, we should convert the event_time into a datatime format

In [32]:
df['event_time'] = pd.to_datetime(df['event_time'])

Furthermore, we are checking how many unique entries we have per row

In [33]:
df.nunique()

event_time       501143
product_id        24255
category_id         260
category_code       104
brand               728
price              9478
user_id          224971
user_session     273421
event_type            3
dtype: int64

In [34]:
df.event_type.value_counts()

event_type
view        458082
cart         34668
purchase     23105
Name: count, dtype: int64

Event Type will be our target column. What we see here is a huge class imbalance. We will need to take care of that before starting the models.

In [35]:
df.groupby('event_type')['user_id'].nunique()

event_type
cart         23158
purchase     13041
view        224741
Name: user_id, dtype: int64

In [36]:
df.groupby('event_type')['user_id'].count()

event_type
cart         34668
purchase     23105
view        458082
Name: user_id, dtype: int64

In [37]:
user_mean_price = df.groupby('user_id')['price'].mean()
user_mean_price.head()

user_id
1515915625353226922     76.480000
1515915625353230683     62.686923
1515915625353230922    274.400000
1515915625353234047    156.220286
1515915625353236157     52.490000
Name: price, dtype: float64

In [38]:
purchasers = df[df['event_type'] == 'purchase'][['user_id']].drop_duplicates()
purchasers_all_events = df[df['user_id'].isin(purchasers['user_id'])]
purchasers_all_events.head()

,event_time,product_id,category_id,category_code,brand,price,user_id,user_session,event_type
34,2020-09-24 12:02:34+00:00,1507291,2144415926806642904,computers.components.power_supply,supermicro,217.57,1515915625519390468,UzabYgCPva,view
37,2020-09-24 12:02:53+00:00,1507291,2144415926806642904,computers.components.power_supply,supermicro,217.57,1515915625519390468,UzabYgCPva,cart
42,2020-09-24 12:03:17+00:00,1507291,2144415926806642904,computers.components.power_supply,supermicro,217.57,1515915625519390468,xn6SHCnZtk,view
45,2020-09-24 12:04:10+00:00,1507291,2144415926806642904,computers.components.power_supply,supermicro,217.57,1515915625519390468,xn6SHCnZtk,purchase
70,2020-09-24 12:12:06+00:00,822426,2144415922360680550,computers.peripherals.camera,logitech,123.35,1515915625513574486,2gngxS29Ts,view


In [39]:
mean_event_type_per_user = df.groupby(['user_id', 'event_type']).size().groupby('event_type').mean()
mean_event_type_per_user

event_type
cart        1.497020
purchase    1.771720
view        2.038266
dtype: float64

In [40]:
df.event_time.min(), df.event_time.max()

(Timestamp('2020-09-24 11:57:26+0000', tz='UTC'),
 Timestamp('2021-02-28 23:59:09+0000', tz='UTC'))

In [41]:
df['event_time'] = pd.to_datetime(df['event_time'])  # Ensure event_time is in datetime format
df['month'] = df['event_time'].dt.to_period('M')  # Extract the month

columns_per_month = df.groupby('month').size()
columns_per_month

/var/folders/pm/ddtx28cd2hn37jlq10mrtz700000gn/T/ipykernel_42857/373949976.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['month'] = df['event_time'].dt.to_period('M')  # Extract the month


month
2020-09     15387
2020-10     88849
2020-11    106635
2020-12     88490
2021-01    115234
2021-02    101260
Freq: M, dtype: int64

In [42]:
df.groupby(['month', 'event_type']).size()

month    event_type
2020-09  cart             834
         purchase         588
         view           13965
2020-10  cart            5123
         purchase        3659
         view           80067
2020-11  cart            6459
         purchase        4544
         view           95632
2020-12  cart            5873
         purchase        3961
         view           78656
2021-01  cart            8652
         purchase        5415
         view          101167
2021-02  cart            7727
         purchase        4938
         view           88595
dtype: int64